In [1]:
# 데이터 가져오기

import json
from urllib.request import urlopen
import pandas as pd
import numpy as np
from datetime import date
date=str(date.today().year)+str(date.today().month)+str(date.today().day)
start_date=str(int(date)-100)
key='U8U9S9MEHH8NU1X1BRBV'
nara=['0000001','0000002','0000003','0000053']   # 나라 코드(0000001:us, 0000002:jp, 0000003:eu, 0000053:cn)
url=["http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/100000/036Y001/DD/20180401/date/0000001/?/?/",  # 환율
    "http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/100000/098Y001/MM/20190401/date/0101000/?/?/",   # 기준금리
    "http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/100000/022Y013/MM/20190401/date/000000/?/?/",    # 경상수지
    ]    

s_data=pd.DataFrame()
time=[]
us=[]
jp=[]
eu=[]
cn=[]

for i in range(len(nara)):
    data=json.loads(urlopen(url[0][:-12]+nara[i]+url[0][-5:]).read())
    data2=data['StatisticSearch']['row']
    for k in range(len(data2)):        
        if nara[i]==nara[0]:
            us.append(float(data2[k]['DATA_VALUE']))
            time.append(int(data2[k]['TIME']))
        elif nara[i]==nara[1]:
            jp.append(float(data2[k]['DATA_VALUE']))
        elif nara[i]==nara[2]:
            eu.append(float(data2[k]['DATA_VALUE']))
        else :
            cn.append(float(data2[k]['DATA_VALUE']))

s_data['time']=time
s_data['us']=us
s_data['jp']=jp
s_data['eu']=eu
s_data['cn']=cn
s_data['time']=pd.to_datetime(s_data.time.apply(lambda x: str(x)))
print(s_data)


          time      us       jp       eu      cn
0   2018-04-02  1064.4  1000.80  1311.07  169.80
1   2018-04-03  1058.1   999.67  1301.73  168.83
2   2018-04-04  1057.8   992.82  1298.08  168.60
3   2018-04-05  1057.6   990.68  1299.31  168.26
4   2018-04-06  1059.5   989.03  1297.41  168.56
..         ...     ...      ...      ...     ...
527 2020-05-25  1236.2  1148.08  1348.01  173.25
528 2020-05-26  1242.0  1152.83  1353.41  173.72
529 2020-05-27  1237.1  1150.63  1358.77  173.00
530 2020-05-28  1233.7  1144.91  1358.92  172.15
531 2020-05-29  1239.4  1150.63  1372.57  172.63

[532 rows x 5 columns]


In [2]:
st_data=s_data[-1:]
s_data=s_data[:-1]
stat=pd.DataFrame()  # us
stat2=pd.DataFrame() # jp
stat3=pd.DataFrame() # eu
stat4=pd.DataFrame() # cn

stat['or']=st_data[['us']].reset_index()['us']
stat2['or']=st_data[['jp']].reset_index()['jp']
stat3['or']=st_data[['eu']].reset_index()['eu']
stat4['or']=st_data[['cn']].reset_index()['cn']


In [3]:
# LSTM 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [ ]:
# LSTM_US
import random as rn
import os
os.environ['PYTHONHASHSEED']=str(0)
np.random.seed(0)
tf.random.set_seed(0)
rn.seed(0)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(s_data[['us']])
scaled_train_data = scaler.transform(s_data[['us']])
scaled_test_data = scaler.transform(st_data[['us']])

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(200, kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit_generator(generator,epochs=20)

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(st_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
stat['lstm']=lstm_predictions
print(lstm_predictions)

Epoch 1/20
530/530 [==============================] - 222s 420ms/step - loss: 1.3475
Epoch 2/20
182/530 [=========>....................] - ETA: 3:16 - loss: 0.9217

In [ ]:
# LSTM_JP

os.environ['PYTHONHASHSEED']=str(0)
np.random.seed(0)
tf.random.set_seed(0)
rn.seed(0)

scaler = MinMaxScaler()
scaler.fit(s_data[['jp']])
scaled_train_data = scaler.transform(s_data[['jp']])
scaled_test_data = scaler.transform(st_data[['jp']])


from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(200, kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit_generator(generator,epochs=20)

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(st_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
stat2['lstm']=lstm_predictions
print(lstm_predictions)

In [ ]:
# LSTM_CN

random.seed(0)
os.environ['PYTHONHASHSEED']=str(0)
np.random.seed(0)
tf.random.set_seed(0)

scaler = MinMaxScaler()
scaler.fit(s_data[['cn']])
scaled_train_data = scaler.transform(s_data[['cn']])
scaled_test_data = scaler.transform(st_data[['cn']])


from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(200,  kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit_generator(generator,epochs=20)

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(st_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
stat4['lstm']=lstm_predictions
print(lstm_predictions)

In [ ]:
print(stat)
print(stat2)
print(stat4)

In [ ]:
# savgol_filter

from scipy.signal import savgol_filter
smoothed_2dg = savgol_filter(s_data['us'], window_length = 5, polyorder = 2)
smoothed_1dg = savgol_filter(s_data['us'], window_length = 5, polyorder = 1)
smoothed_2dg=smoothed_2dg.reshape(-1,1)
smoothed_1dg=smoothed_1dg.reshape(-1,1)

# LSTM_US_savgol_filter
os.environ['PYTHONHASHSEED']=str(0)
np.random.seed(0)
tf.random.set_seed(0)
rn.seed(0)
scaler = MinMaxScaler()
scaler.fit(smoothed_1dg)
scaled_train_data = scaler.transform(smoothed_1dg)
scaled_test_data = scaler.transform(smoothed_1dg)

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(200, kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit_generator(generator,epochs=10)

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(st_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
print(lstm_predictions)